## **실행 전에**

reversed_data_after_preprocessing.csv 추가

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.getcwd()
os.chdir("/content/drive/My Drive/Colab Notebooks/mecab")

In [3]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.


In [4]:
cd Mecab-ko-for-Google-Colab/

/content/drive/My Drive/Colab Notebooks/mecab/Mecab-ko-for-Google-Colab


In [5]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 3.8MB 37.8MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-08-30 11:08:27--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.0, 18.205.93.1, 18.205.93.2, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.0|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?S

# 전처리

In [6]:
import pandas as pd
import csv
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Mecab

In [41]:
data = pd.read_csv('../reversed_data_after_preprocessing.csv')
data

,title,content
0,서해상으로 북상 중인 태풍 바비 피해 줄이려면,Q 특히 농어촌에서 어떤 피해가 발생할지가 걱정인데요 Q 도심 집에 계신 ...
1,새벽 4시 5시 사이 서울 근접 이 시각 서울시 재난안전상황실,만약 이런 피해가 발생하면 가까운 구청 상황실이나 다산콜센터 등에 연락하시면 됩니...
2,코로나 때문에 봉사시간 못 채워 졸업 막힌 대학생들,정부 차원의 대책 마련이 필요하다는 의견도 나온다 코로나19라는 비상 상황인 만...
3,성인 남자가 서 있기 힘들 정도 태풍 바비 군산 도착,지금까지 전북 군산 비응항에서 YTN 김민성 입니다 이번 태풍 아무쪼록 큰 피해 ...
4,위험 반원 인천 초비상 초속 25m 넘으면 영종대교 통제,또 초속 25m 이상 바람이 불 경우 인천공항으로 향하는 영종대교와 인천대교 차량 ...
...,...,...
9370,깜깜이 감염 확진자 확산하는 원주시 다중이용시설 폐쇄,시는 전문 역학조사팀이 CCTV 확인과 카드 사용 조사 등 정밀한 역학조사를...
9371,원주 코로나19 확진 잇달아 5명 추가 확진,도교육청은 오늘 원주교육지원청에서 원주권 유치원 초 중 고 특수학교 전체 비상교...
9372,경기 어제 신규확진 109명 코로나사태 이후 일일 최다,신규 확진자 중 감염 경로가 확인되지 않아 조사 중 으로 분류된 환자가 2...
9373,방역당국 사랑제일교회 밤샘대치 21일 명단확보 재시도 위해 대기 중,현재 방역당국 관계자들은 오전 9시 현재까지 현장에 남아 역학조사 재개를 위한 대기...


토큰화 + 불용어제거

In [42]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다',
           '인데요','다','을','습니다','Q','이다','보인다','한다','했다','였다','이다']

In [43]:
encoder_input, decoder_input, decoder_output = [], [], []

content_tokenizer = Mecab()
title_tokenizer = Mecab()

for line in data['content']:
    temp = content_tokenizer.morphs(str(line))
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    encoder_input.append(temp)

for line in data['title']:
    temp = []
    temp.append('<start>')
    for word in title_tokenizer.morphs(str(line)):
      if word not in stopwords: # 불용어 제거
        temp.append(word) 
    decoder_input.append(temp)
  
for line in data['title']:
    temp = []
    for word in title_tokenizer.morphs(str(line)):
      if word not in stopwords: # 불용어 제거
        temp.append(word) 
    temp.append('<end>')
    decoder_output.append(temp)

In [44]:
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_output[:3])

[['특히', '농어촌', '에서', '어떤', '피해', '발생', '할지', '걱정', '도심', '집', '계신', '분', '어떻게', '대비', '해야', '합니까', '특히', '고층', '아파트', '유리창', '깨지', '피해', '태풍', '때', '종종', '발생', '했잖아요', '그때', '신문지', '붙인다', '든가', '테이프', '붙여', '놓', '으면', '예방', '되', '나요', '요즘', '창틀', '창틀', '사이', '헐거운', '부분', '고정', '하', '게', '중요', '하', '다고', '하', '던데', '태풍', '한창', '일', '때', '운전', '하', '지', '않', '게', '가장', '좋', '겠', '만', '어쩔', '수', '없이', '운전', '해야', '되', '경우', '있', '잖아요', '이럴', '때', '어떻게', '조심', '해야', '됩니까', '도심', '강풍', '피해', '로', '대표', '적', '간판', '떨어진다', '든가', '교회', '첨탑', '무너지', '경우', '있', '었', '이렇게', '서해', '상', '태풍', '올라오', '면', '주로', '어떤', '피해', '발생', '하', '나요', '이번', '바람', '피해', '강할', '것', '같', '강원', '대학교', '방재', '전문', '대학원', '김병식', '교수', '님', '모시', '고', '태풍', '피해', '어떻게', '대응', '해야', '하', '는지', '자세', '얘기', '나눠', '보', '겠'], ['만약', '이런', '피해', '발생', '하', '면', '가까운', '구청', '상황실', '이나', '다산', '콜', '센터', '등', '연락', '하', '시', '면', '됩니다', '또', '정전', '피', '해도', '있', '수', '있', '공사장', '가림막', '이나', '현수막', '건물', '간판', '아파트', '유리창',

정수인코딩

In [50]:
import nltk
from nltk import FreqDist
import numpy as np

encoder_vocab = FreqDist(np.hstack(encoder_input))
decoder_vocab = FreqDist(np.append(np.hstack(decoder_input), ['<end>'] * len(decoder_output)))

print(len(encoder_vocab))
print(len(decoder_vocab))

40125
9714


In [51]:
# 빈도수 적게 나온 단어 빼기
encoder_vocab = [(morph,freq) for morph, freq in encoder_vocab.most_common() if freq > 3]
decoder_vocab = [(morph,freq) for morph, freq in decoder_vocab.most_common() if freq > 3]

print(len(encoder_vocab))
print(len(decoder_vocab))

20110
3451


In [53]:
encoder_word_to_index = {word[0]: index + 2 for index, word in enumerate(encoder_vocab)}
decoder_word_to_index = {word[0]: index + 2 for index, word in enumerate(decoder_vocab)}

In [54]:
len(decoder_word_to_index)

3451

In [55]:
# 특수 인덱스
encoder_word_to_index['unk'] = 1

encoder_encoded_input = []
for line in encoder_input:
    tmp = []
    for word in line:
        try:
            # 각 글자를 해당하는 정수로 변환한다.
            tmp.append(encoder_word_to_index[word])
        except KeyError: 
            # 단어 집합에 없는 단어일 경우 unk로 대체된다.
            tmp.append(encoder_word_to_index['unk'])

    encoder_encoded_input.append(tmp)

# 기존의 리뷰가 성공적으로 encoding 되었는지 확인해보기
print(encoder_input[1])
print(encoder_encoded_input[1])

['만약', '이런', '피해', '발생', '하', '면', '가까운', '구청', '상황실', '이나', '다산', '콜', '센터', '등', '연락', '하', '시', '면', '됩니다', '또', '정전', '피', '해도', '있', '수', '있', '공사장', '가림막', '이나', '현수막', '건물', '간판', '아파트', '유리창', '등', '바람', '날아가', '면', '사람', '다칠', '수', '있', '는데요', '외출', '하', '지', '않', '게', '가장', '좋', '고', '부득이', '하', '게', '하', '실', '경우', '엔', '오래', '된', '건물', '이나', '공사장', '주변', '피하', '시', '게', '좋', '다만', '강풍', '인한', '피해', '있', '수', '있', '어', '대비', '하', '셔야', '하', '는데요', '서울시', '신호등', '이나', '가로등', '같', '도로', '교통', '시설물', '이나', '가로수', '강풍', '쓰러질', '수', '있', '다고', '경고', '했', '태풍', '지나가', '면', '시민', '각별히', '조심', '해야', '할', '텐데요', '서울', '시내', '에서', '태풍', '취약', '지점', '어디', '입니까', '네', '현재', '레이더', '영상', '보', '시', '면', '태풍', '지나가', '오늘', '새벽', '서울', '비', '많이', '예보', '돼', '있', '지', '않', '또', '서울시', '공공', '자전거', '인', '따', '릉', '운행', '어제', '저녁', '6', '시', '부터', '태풍', '소멸', '할', '때', '까지', '중단', '합니다', '서울시', '25', '개', '자치구', '있', '코로나', '19', '실외', '선별', '진료소', '오늘', '오전', '9', '시', '부터', '오후', '1', '시', '

In [56]:
# 단어들에 순차적으로 2 ~ 까지의 인덱스를 부여한다(unknown=1)
decoder_word_to_index['unk'] = 1

decoder_encoded_input = []
for line in decoder_input:
    tmp = []
    for word in line:
        try:
            tmp.append(decoder_word_to_index[word])
        except KeyError: 
            tmp.append(decoder_word_to_index['unk'])

    decoder_encoded_input.append(tmp)


print(decoder_input[1])
print(decoder_encoded_input[1])

['<start>', '새벽', '4', '시', '5', '시', '사이', '서울', '근접', '시각', '서울시', '재난', '안전', '상황실']
[2, 588, 65, 77, 49, 77, 769, 27, 347, 548, 130, 160, 515, 1]


In [57]:
decoder_encoded_output = []
for line in decoder_output:
    tmp = []
    for word in line:
        try:
            tmp.append(decoder_word_to_index[word])
        except KeyError: 
            tmp.append(decoder_word_to_index['unk'])

    decoder_encoded_output.append(tmp)



print(decoder_output[1])
print(decoder_encoded_output[1])

['새벽', '4', '시', '5', '시', '사이', '서울', '근접', '시각', '서울시', '재난', '안전', '상황실', '<end>']
[588, 65, 77, 49, 77, 769, 27, 347, 548, 130, 160, 515, 1, 3]


In [58]:
# 기존의 리뷰가 성공적으로 encoding 되었는지 확인해보기
print(encoder_encoded_input[0])
print(decoder_encoded_input[0])
print(decoder_encoded_output[0])

[290, 6111, 7, 648, 144, 59, 1565, 741, 1579, 319, 3648, 123, 719, 449, 146, 7282, 290, 8667, 686, 3598, 5512, 144, 71, 131, 4900, 59, 11185, 1861, 7572, 1, 9426, 4546, 5796, 673, 441, 740, 23, 2435, 2344, 5435, 5435, 457, 1, 473, 2174, 2, 30, 539, 2, 52, 2, 7751, 71, 4184, 6, 131, 2355, 2, 15, 27, 30, 257, 507, 69, 37, 2997, 19, 410, 2355, 146, 23, 112, 4, 2138, 5008, 131, 719, 2230, 146, 8083, 1579, 454, 144, 9, 330, 14, 1971, 9163, 9426, 32, 7146, 3145, 112, 4, 40, 568, 999, 231, 71, 2345, 25, 1397, 648, 144, 59, 2, 2435, 166, 313, 144, 5572, 11, 115, 700, 1367, 6031, 915, 2046, 1, 317, 551, 6112, 3, 71, 144, 719, 288, 146, 2, 479, 1543, 885, 2009, 87, 69]
[2, 868, 113, 144, 29, 114, 17, 40, 76, 2003, 2219]
[868, 113, 144, 29, 114, 17, 40, 76, 2003, 2219, 3]


패딩

In [61]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# maxlen 없어도 알아서 잘 패딩합니다

encoder_input = pad_sequences(encoder_encoded_input, padding="post")
decoder_input = pad_sequences(decoder_encoded_input, padding="post")
decoder_output = pad_sequences(decoder_encoded_output, padding="post")

In [62]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_output.shape)
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_output[:3])

(9375, 5365)
(9375, 25)
(9375, 25)
[[ 290 6111    7 ...    0    0    0]
 [1382  275  144 ...    0    0    0]
 [  58 1238  194 ...    0    0    0]]
[[   2  868  113  144   29  114   17   40   76 2003 2219    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [   2  588   65   77   49   77  769   27  347  548  130  160  515    1
     0    0    0    0    0    0    0    0    0    0    0]
 [   2    6  470 2529  240   75    1 2924 2530 2531    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
[[ 868  113  144   29  114   17   40   76 2003 2219    3    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [ 588   65   77   49   77  769   27  347  548  130  160  515    1    3
     0    0    0    0    0    0    0    0    0    0    0]
 [   6  470 2529  240   75    1 2924 2530 2531    3    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]


테스트-트레인셋 분리

In [64]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[6839  439 5571 ... 8303 7112 9096]


In [76]:
# 3:1로 나눴습니다
test_size = int(encoder_input.shape[0] * 0.25)
train_size = encoder_input.shape[0] - test_size
train_idx = indices[:train_size]
test_idx = indices[train_size:]
encoder_input_train = encoder_input[train_idx]
decoder_input_train = decoder_input[train_idx]
decoder_output_train = decoder_output[train_idx]

encoder_input_test = encoder_input[test_idx]
decoder_input_test = decoder_input[test_idx]
decoder_output_test = decoder_output[test_idx]

In [77]:
print(encoder_input_train.shape)
print(encoder_input_test.shape)

(7032, 5365)
(2343, 5365)


# 학습

In [79]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

인코더

In [86]:
encoder_inputs = Input(shape=(encoder_input_train.shape[1],))
encoder_embed = Embedding(len(encoder_word_to_index)+1, 50)(encoder_inputs)
encoder_mask = Masking(mask_value=0)(encoder_embed)
encoder_outputs, h_state, c_state = LSTM(50, return_state=True)(encoder_mask)

디코더

In [87]:
decoder_inputs = Input(shape=(decoder_input_train.shape[1],))
decoder_embed = Embedding(len(decoder_word_to_index)+1, 50)(decoder_inputs)
decoder_mask = Masking(mask_value=0)(decoder_embed)

decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_mask, initial_state=[h_state, c_state])
decoder_dense = Dense(len(decoder_word_to_index)+1, activation='softmax')
decoder_softmax_outputs = decoder_dense(decoder_outputs)

In [88]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# validation loss를 계속 보다가 5회 이상 loss가 증가하면, 과적합될 수 있으므로 학습을 조기 종료하겠다.
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=8)
# epoch를 반복하면서, 가장 검증데이터 정확도가 높았던 순간을 체크포인트로 저장
# 정확도가 낮아지면 모델 버려라
model_check = ModelCheckpoint('mini_project_model_01.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_output_train, 
          validation_data = ([encoder_input_test, decoder_input_test], decoder_output_test), 
          batch_size = 128, epochs = 50, callbacks=[early_stop, model_check])

Epoch 1/50
55/55 [==============================] - ETA: 0s - loss: 2.7174 - acc: 0.6046
Epoch 00001: val_acc improved from 0.60120 to 0.60270, saving model to mini_project_model_01.h5
55/55 [==============================] - 503s 9s/step - loss: 2.7174 - acc: 0.6046 - val_loss: 2.7940 - val_acc: 0.6027
Epoch 2/50
26/55 [=============>................] - ETA: 4:01 - loss: 2.6975 - acc: 0.6056

# 예측

In [90]:
encoder_model = Model(encoder_inputs, [h_state, c_state])

encoder_h_state = Input(shape=(50,))
encoder_c_state = Input(shape=(50,))
pd_decoder_outputs, pd_h_state, pd_c_state = decoder_lstm(decoder_mask, initial_state=[encoder_h_state, encoder_c_state])
pd_decoder_softmax_outputs = decoder_dense(pd_decoder_outputs)

decoder_model = Model([decoder_inputs] + [encoder_h_state, encoder_c_state], [pd_decoder_softmax_outputs] + [pd_h_state, pd_c_state])

In [96]:
index_to_title = {index: title for title, index in decoder_word_to_index.items()}

In [118]:
# def seq2summary(input_content_str): # input : 전처리된 content(str)
#     temp_text = content_tokenizer.morphs(input_content_str)
#     temp_text = [word for word in temp_text if not word in stopwords]
    
#     temp_encoding = []
#     for word in temp_text:
#         temp_encoding.append(encoder_word_to_index[word])

#     temp_encoding = pad_sequences(temp_encoding, padding='post', max_len=encoder_input.shape[1])  
    
#     return temp_encoding # output : 정수 인코딩 된 리스트


def decode_sequence(encoder_input): 
    states_value = encoder_model.predict(encoder_input)
    
    predicted_seq = np.zeros((1, 1))
    predicted_seq[0, 0] = decoder_word_to_index['<start>']
    
    decoded_stc = []
    while True:
        output_words, h, c = decoder_model.predict([predicted_seq] + states_value)
        predicted_word = index_to_title[np.argmax(output_words[0, 0])]

        if predicted_word == '<end>' or len(decoded_stc.split()) >= decoder_input_train.shape[1]:
            break

        decoded_stc.append(predicted_word)
        
        predicted_seq = np.zeros((1, 1))
        predicted_seq[0, 0] = np.argmax(output_words[0, 0])
        states_value = [h, c]

    return ' '.join(decoded_stc)   # 예측된 title (str)

In [135]:
[predicted_seq]

[array([[2.]])]

In [136]:
a = encoder_input_test[0]
states_value = encoder_model.predict(a)

predicted_seq = np.zeros((1, 1))
predicted_seq[0, 0] = decoder_word_to_index['<start>']

decoded_stc = []
while True:
    output_words, h, c = decoder_model.predict(predicted_seq + states_value)
    predicted_word = index_to_title[np.argmax(output_words[0, 0])]

    if predicted_word == '<end>' or len(decoded_stc.split()) >= decoder_input_train.shape[1]:
        break

    decoded_stc.append(predicted_word)

    predicted_seq = np.zeros((1, 1))
    predicted_seq[0, 0] = np.argmax(output_words[0, 0])
    states_value = [h, c]


AssertionError: ignored

In [106]:
# test_data = data.iloc[test_idx, :]

# test_data['title_predicted'] = test_data['content'].map(lambda x: decode_sequence(x))

# test_data

NameError: ignored